## 1. Setup and Authentication

We start by importing necessary libraries and configure authentication with a GitHub personal access token.

In [2]:
# Libraries
import requests
import time
import json

In [3]:
# Set the GitHub personal access token
GITHUB_TOKEN = "" (Add personal GitHub token here)
HEADERS = {"Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"}

## 2. Search Public Repositories

We will use the `/search/repositories` endpoint to find public repositories related to marketing and data.

In [17]:
def search_repositories(query, max_pages, per_page=10): #This search includes pagination to select te desired amount of pages
    all_results = []
    for page in range(1, max_pages + 1):
        url = "https://api.github.com/search/repositories"
        params = {"q": query,
            "per_page": per_page,
            "page": page}
        response = requests.get(url, headers=HEADERS, params=params)
        data = response.json()
        items = data.get("items", [])
        all_results.extend(items)

        if len(items) < per_page:  # Break early if there are no more results
            break
            time.sleep(1)  # avoid rate limit if looping many times
        return all_results

In [18]:
#Search for marketing data tools
repos = search_repositories("marketing analytics", max_pages=3)
for repo in repos:
    print(f"{repo['full_name']} - ⭐ {repo['stargazers_count']}")


plausible/analytics - ⭐ 22725
TrainingByPackt/Data-Science-for-Marketing-Analytics - ⭐ 217
Nikhilkohli1/Digital-Marketing-Analytics - ⭐ 231
GoogleCloudPlatform/marketing-analytics-jumpstart - ⭐ 158
PacktPublishing/Data-Science-for-Marketing-Analytics-Second-Edition - ⭐ 128
retentioneering/retentioneering-tools - ⭐ 839
byukan/Marketing-Data-Science - ⭐ 293
GoogleCloudPlatform/marketing-analytics-jumpstart-dataform - ⭐ 70
blurred-machine/Sales-and-Marketing-Analytics - ⭐ 74
Phippsy/R-Marketing-Analytics - ⭐ 33


## 3. List Commits in the selected Repository

Extract commit activity from a repository using the `/repos/{owner}/{repo}/commits` endpoint.

In [6]:
def list_commits(owner, repo, per_page=5):
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    params = {"per_page": per_page}
    response = requests.get(url, headers=HEADERS, params=params)
    return response.json()

In [10]:
# Get commits from the plausible/analytics repo (Top from section 2)
commits = list_commits("plausible", "analytics")
for c in commits:
    print(c['commit']['author']['date'], "-", c['commit']['message'])

2025-06-24T16:11:27Z - Calmer multiple teams experience (#5526)

* Offer team switcher on /sites if applicable

- in case of empty My Personal Sites view, and with
  another team with sites being available
- redirect straight to first team upon invoking team
  switcher, if there's only one available
- redirect to /sites from team switcher, if there
  are no set-up teams available

* Remove unused test helper

* Store and use last team identifier

* Remove alert about starting trial when adding first site

* Format

* Update lib/plausible_web/live/sites.ex

Co-authored-by: Adrian Gruntkowski <adrian.gruntkowski@gmail.com>

---------

Co-authored-by: Adrian Gruntkowski <adrian.gruntkowski@gmail.com>
2025-06-24T15:02:25Z - Migration: Add `users.last_team_identifier` (#5527)
2025-06-24T11:51:32Z - Add "SSO" billing plan feature and hide SSO setup behind it (#5524)

* Add SSO plan feature

* Gate SSO configuration with plan feature

* Don't preselect SSO feature when creating new custom pla

## 4. Get File Contents from a Repository

Use the `/repos/{owner}/{repo}/contents/{path}` endpoint to read a specific file or folder in a repo.


In [12]:
def get_contents(owner, repo, path="README.md"):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    response = requests.get(url, headers=HEADERS)
    return response.json()

In [13]:
# Get the README file from dbt-core
content = get_contents("plausible", "analytics")
print(content.get("name"), "-", content.get("type"), "-", content.get("html_url"))

README.md - file - https://github.com/plausible/analytics/blob/master/README.md


## 5. Rate Limit and Error Handling

In this last section we handle diffrerent errors like 401 Unauthorized and check GitHub’s rate limits.

In [14]:
def check_rate_limit():
    url = "https://api.github.com/rate_limit"
    response = requests.get(url, headers=HEADERS)
    data = response.json()
    rate = data.get('rate', {})
    print("Remaining requests:", rate.get('remaining'))
    print("Reset time (UTC):", time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(rate.get('reset'))))

check_rate_limit()

Remaining requests: 4997
Reset time (UTC): 2025-06-24 20:19:44
